
# Chatbot IA avec Rag

Ce notebook implémente un chatbot IA avancé en utilisant **LangChain** et **LangGraph**.  
Il intègre :
- **RAG (Retrieval-Augmented Generation) avec affichage des sources**
- **Streaming des réponses pour une meilleure expérience utilisateur**
- **Mémoire conversationnelle pour un suivi des interactions**


In [3]:

!pip install --upgrade --quiet langchain langchain-community langchainhub beautifulsoup4 langchain-openai beautifulsoup4 chromadb
!pip install --quiet langgraph
!pip install --upgrade langchain-mistralai



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


##  Importation des bibliothèques

In [7]:
#  Importation des bibliothèques
import os
import json
import getpass

from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

## Connexion à l'Api et initialisation du modèle

In [8]:
# ----------------------------
# Configuration API Mistral AI
# ----------------------------
MISTRAL_API_KEY = getpass.getpass("Entrez votre clé API Mistral: ")  
MODEL_NAME = "mistral-small-latest" #test du modèle small pour combiner performance et prix accessible


In [9]:
# Initialisation du modèle / activation du streaming
llm = ChatMistralAI(
    mistral_api_key=MISTRAL_API_KEY,
    model=MODEL_NAME,
    streaming=True  
)


In [ ]:
# -----------------------------
# Chargement de la Vector DB Chroma
# -----------------------------
CHROMA_DB_PATH = "./chroma_langchain_db" 
vector_store = Chroma(persist_directory=CHROMA_DB_PATH)
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3})


##  Définition du Prompt

In [ ]:
# ----------------------------
# Définition du Prompt Optimisé
# ----------------------------
prompt_template = ChatPromptTemplate.from_template(
    """
    Tu es un conseiller d'orientation professionnelle expérimenté, spécialisé dans l'accompagnement personnalisé des lycéens et étudiants. Ton rôle est d'aider l'utilisateur à faire des choix éclairés de formation ou de carrière en fonction de son parcours, de ses compétences, de ses hobbies et de ses aspirations professionnelles.

    CONTEXTE:
    ---------------------
    {context}
    ---------------------

    Question de l'utilisateur: {input}

    Fournis une réponse claire, structurée et motivante, en proposant des formations ou métiers adaptés, et en expliquant en quoi ils correspondent aux informations fournies par l'utilisateur.
    """
)
"""

##  Définition du Chatbot avec mémoire, RAG

In [ ]:
# ----------------------------
# Mise en place de la mémoire conversationnelle
# ----------------------------
memory = MemorySaver()
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    """Appelle le modèle avec historique de conversation"""
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
chatbot = workflow.compile(checkpointer=memory)


In [ ]:

# ----------------------------
# Fonction pour générer une réponse avec sources (RAG)
# ----------------------------
def generate_response(user_query: str):
    """ Génère une réponse optimisée en incluant les sources RAG."""

    # Récupération du contexte
    retrieved_docs = retriever.invoke(user_query)
    context = "\n".join([f"Source: {doc.metadata.get('source', 'Inconnu')}\n{doc.page_content}" for doc in retrieved_docs])

    # Construction du prompt
    formatted_prompt = prompt_template.format(input=user_query, context=context)

    # Appel au modèle Mistral
    response = llm.invoke(formatted_prompt)

    return {
        "response": response,
        "sources": [doc.metadata.get("source", "Inconnu") for doc in retrieved_docs]
    }

## Test du Chatbot

In [ ]:

# ----------------------------
# Test du chatbot avec une question utilisateur
# ----------------------------
user_input = "Je suis étudiant en informatique et j'hésite entre devenir développeur ou data scientist. Que me recommandes-tu ?"
result = generate_response(user_input)

print("\n Réponse du chatbot :")
print(result["response"])

print("\n Sources utilisées :")
for src in result["sources"]:
    print(f" - {src}")
    
